<a href="https://colab.research.google.com/github/eneskosar/Human-Activity-Recognition/blob/master/PreprocessingV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

# Imports

In [ ]:
from __future__ import print_function
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os
import seaborn as sns
from scipy import stats
from IPython.display import display, HTML
import h5py
import time

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, LSTM
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.utils import np_utils

# Function Definitions

In [ ]:
 direct = '/content/gdrive/My Drive/Colab Notebooks/HARproject'

 def create_segments(subjectList ):
    
     N_FEATURES = 45
     time_steps = 125

     segmentsTraining = []
     labelsTraining = []
     subjectsTraining = []

     for activity in range(19):
       if activity < 9:
         file_name = f"{direct}/data/a0{activity+1}"
       else:  
         file_name = f"{direct}/data/a{activity+1}"
       for subject in subjectList:
         file_name_2 = f"{file_name}/p{subject}"
         for segment in range(60):
           if segment <9:
             file_name_3 = f"{file_name_2}/s0{segment+1}.txt"
           else:
             file_name_3 = f"{file_name_2}/s{segment+1}.txt" 
           arr = np.loadtxt(fname = file_name_3, delimiter = ",") # read text file
           segmentsTraining.append([arr])
           labelsTraining.append(activity)
           subjectsTraining.append(subject)

     reshaped_segments = np.asarray(segmentsTraining, dtype = np.float32).reshape(-1, time_steps, N_FEATURES)
     labels = np.asarray(labelsTraining)
     subjects = np.asarray(subjectsTraining)
    
     return reshaped_segments, labels

In [ ]:
 def shuffle_data(segments, labels):
    
     idx = np.random.permutation(segments.shape[0])
     segments_shuffled = segments[idx]
     labels_shuffled = labels[idx]
    
     return segments_shuffled, labels_shuffled

In [ ]:
def normalize_segments(segments):
    normalized_segments = []

    for i in range(segments.shape[0]):
        seg = segments[i, :, :]
        seg_norm = (seg - seg.min(axis=0)) / (seg.max(axis=0) - seg.min(axis=0))
        normalized_segments.append(seg_norm)

    return np.asarray(normalized_segments)


def prepare_data_for_network(data, labels):
    data = data.astype('float32')
    labels = labels.astype('float32')

    labels_hot = np_utils.to_categorical((labels))

    return data, labels_hot

# Variable Declarations

In [ ]:
segments_Training1, segments_Training2, segments_Training3, segments_Training4 = [],[],[],[]
segments_Training5, segments_Training6, segments_Training7, segments_Training8 = [],[],[],[]
labels_Training1, labels_Training2, labels_Training3, labels_Training4 = [],[],[],[]
labels_Training5, labels_Training6, labels_Training7, labels_Training8 = [],[],[],[]

segments_Validation1, segments_Validation2, segments_Validation3, segments_Validation4 = [],[],[],[]
segments_Validation5, segments_Validation6, segments_Validation7, segments_Validation8 = [],[],[],[]
labels_Validation1, labels_Validation2, labels_Validation3, labels_Validation4 = [],[],[],[]
labels_Validation5, labels_Validation6, labels_Validation7, labels_Validation8 = [],[],[],[]

segments = {"segments_Training1": segments_Training1, "labels_Training1":labels_Training1,
            "segments_Training2": segments_Training2, "labels_Training2":labels_Training2,
            "segments_Training3": segments_Training3, "labels_Training3":labels_Training3,
            "segments_Training4": segments_Training4, "labels_Training4":labels_Training4,
            "segments_Training5": segments_Training5, "labels_Training5":labels_Training5,
            "segments_Training6": segments_Training6, "labels_Training6":labels_Training6,
            "segments_Training7": segments_Training7, "labels_Training7":labels_Training7,
            "segments_Training8": segments_Training8, "labels_Training8":labels_Training8,
            "segments_Validation1": segments_Validation1, "labels_Validation1":labels_Validation1,
            "segments_Validation2": segments_Validation2, "labels_Validation2":labels_Validation2,
            "segments_Validation3": segments_Validation3, "labels_Validation3":labels_Validation3,
            "segments_Validation4": segments_Validation4, "labels_Validation4":labels_Validation4,
            "segments_Validation5": segments_Validation5, "labels_Validation5":labels_Validation5,
            "segments_Validation6": segments_Validation6, "labels_Validation6":labels_Validation6,
            "segments_Validation7": segments_Validation7, "labels_Validation7":labels_Validation7,
            "segments_Validation8": segments_Validation8, "labels_Validation8":labels_Validation8}

  

# Applying Functions

In [ ]:
start_time = time.time()

subject_number_list = [1,2,3,4,5,6,7,8]                          
for i in range(8):
  temp = subject_number_list.copy()
  temp.remove(i+1)
  segments[f"segments_Training{i+1}"], segments[f"labels_Training{i+1}"] = create_segments(temp)
#for i in range(8):
  segments[f"segments_Validation{i+1}"], segments[f"labels_Validation{i+1}"] = create_segments([i+1])

end_time = time.time()
print(f"total time is: {end_time-start_time}")

total time is: 367.2416422367096


In [ ]:
start_time = time.time()

for i in range(8):
  segments[f"segments_Training{i+1}"] = normalize_segments(segments[f"segments_Training{i+1}"])
  segments[f"segments_Validation{i+1}"] = normalize_segments(segments[f"segments_Validation{i+1}"])

end_time = time.time()
print(f"total time is: {end_time-start_time}")  


total time is: 6.435402870178223


In [ ]:
start_time = time.time()

for i in range(8):
  print(segments[f"labels_Training{i+1}"].shape)
  segments[f"segments_Training{i+1}"], segments[f"labels_Training{i+1}"] = prepare_data_for_network(segments[f"segments_Training{i+1}"], segments[f"labels_Training{i+1}"])

  segments[f"segments_Validation{i+1}"], segments[f"labels_Validation{i+1}"] = prepare_data_for_network(segments[f"segments_Validation{i+1}"]
  , segments[f"labels_Validation{i+1}"])

end_time = time.time()
print(f"total time is: {end_time-start_time}")

(7980,)
(7980,)
(7980,)
(7980,)
(7980,)
(7980,)
(7980,)
(7980,)
total time is: 0.5576686859130859


In [ ]:
start_time = time.time()

for i in range(8):
  segments[f"segments_Training{i+1}"], segments[f"labels_Training{i+1}"] = shuffle_data(segments[f"segments_Training{i+1}"]
  , segments[f"labels_Training{i+1}"])

  segments[f"segments_Validation{i+1}"], segments[f"labels_Validation{i+1}"] = shuffle_data(segments[f"segments_Validation{i+1}"]
  , segments[f"labels_Validation{i+1}"])

end_time = time.time()
print(f"total time is: {end_time-start_time}")

total time is: 0.4792478084564209


# Variable Declarations

In [ ]:
allDataTraining1, allDataTraining2, allDataTraining3, allDataTraining4 = [],[],[],[]
allDataTraining5,allDataTraining6,allDataTraining7,allDataTraining8 = [],[],[],[]

chestDataTraining1, chestDataTraining2, chestDataTraining3, chestDataTraining4 = [],[],[],[]
chestDataTraining5,chestDataTraining6,chestDataTraining7,chestDataTraining8 = [],[],[],[]

rArmDataTraining1, rArmDataTraining2, rArmDataTraining3, rArmDataTraining4 = [],[],[],[]
rArmDataTraining5,rArmDataTraining6,rArmDataTraining7,rArmDataTraining8 = [],[],[],[]

lArmDataTraining1, lArmDataTraining2, lArmDataTraining3, lArmDataTraining4 = [],[],[],[]
lArmDataTraining5,lArmDataTraining6,lArmDataTraining7,lArmDataTraining8 = [],[],[],[]

rLegDataTraining1, rLegDataTraining2, rLegDataTraining3, rLegDataTraining4 = [],[],[],[]
rLegDataTraining5,rLegDataTraining6,rLegDataTraining7,rLegDataTraining8 = [],[],[],[]

lLegDataTraining1, lLegDataTraining2, lLegDataTraining3, lLegDataTraining4 = [],[],[],[]
lLegDataTraining5,lLegDataTraining6,lLegDataTraining7,lLegDataTraining8 = [],[],[],[]

chest_lArm_lLegDataTraining1, chest_lArm_lLegDataTraining2, chest_lArm_lLegDataTraining3, chest_lArm_lLegDataTraining4 = [],[],[],[]
chest_lArm_lLegDataTraining5,chest_lArm_lLegDataTraining6,chest_lArm_lLegDataTraining7,chest_lArm_lLegDataTraining8 = [],[],[],[]
##########################################################
allDataValidation1, allDataValidation2, allDataValidation3, allDataValidation4 = [],[],[],[]
allDataValidation5,allDataValidation6,allDataValidation7,allDataValidation8 = [],[],[],[]

chestDataValidation1, chestDataValidation2, chestDataValidation3, chestDataValidation4 = [],[],[],[]
chestDataValidation5,chestDataValidation6,chestDataValidation7,chestDataValidation8 = [],[],[],[]

rArmDataValidation1, rArmDataValidation2, rArmDataValidation3, rArmDataValidation4 = [],[],[],[]
rArmDataValidation5,rArmDataValidation6,rArmDataValidation7,rArmDataValidation8 = [],[],[],[]

lArmDataValidation1, lArmDataValidation2, lArmDataValidation3, lArmDataValidation4 = [],[],[],[]
lArmDataValidation5,lArmDataValidation6,lArmDataValidation7,lArmDataValidation8 = [],[],[],[]

rLegDataValidation1, rLegDataValidation2, rLegDataValidation3, rLegDataValidation4 = [],[],[],[]
rLegDataValidation5,rLegDataValidation6,rLegDataValidation7,rLegDataValidation8 = [],[],[],[]

lLegDataValidation1, lLegDataValidation2, lLegDataValidation3, lLegDataValidation4 = [],[],[],[]
lLegDataValidation5,lLegDataValidation6,lLegDataValidation7,lLegDataValidation8 = [],[],[],[]

chest_lArm_lLegDataValidation1, chest_lArm_lLegDataValidation2, chest_lArm_lLegDataValidation3, chest_lArm_lLegDataValidation4 = [],[],[],[]
chest_lArm_lLegDataValidation5,chest_lArm_lLegDataValidation6,chest_lArm_lLegDataValidation7,chest_lArm_lLegDataValidation8 = [],[],[],[]



In [ ]:
particular_sensor_data = {
    
            "allDataTraining1": allDataTraining1,
            "allDataTraining2": allDataTraining2,
            "allDataTraining3": allDataTraining3,
            "allDataTraining4": allDataTraining4,
            "allDataTraining5": allDataTraining5,
            "allDataTraining6": allDataTraining6,
            "allDataTraining7": allDataTraining7,
            "allDataTraining8": allDataTraining8,

            "chestDataTraining1": chestDataTraining1,
            "chestDataTraining2": chestDataTraining2,
            "chestDataTraining3": chestDataTraining3,
            "chestDataTraining4": chestDataTraining4,
            "chestDataTraining5": chestDataTraining5,
            "chestDataTraining6": chestDataTraining6,
            "chestDataTraining7": chestDataTraining7,
            "chestDataTraining8": chestDataTraining8,

            "rArmDataTraining1": rArmDataTraining1,
            "rArmDataTraining2": rArmDataTraining2,
            "rArmDataTraining3": rArmDataTraining3,
            "rArmDataTraining4": rArmDataTraining4,
            "rArmDataTraining5": rArmDataTraining5,
            "rArmDataTraining6": rArmDataTraining6,
            "rArmDataTraining7": rArmDataTraining7,
            "rArmDataTraining8": rArmDataTraining8, 

            "lArmDataTraining1": lArmDataTraining1,
            "lArmDataTraining2": lArmDataTraining2,
            "lArmDataTraining3": lArmDataTraining3,
            "lArmDataTraining4": lArmDataTraining4,
            "lArmDataTraining5": lArmDataTraining5,
            "lArmDataTraining6": lArmDataTraining6,
            "lArmDataTraining7": lArmDataTraining7,
            "lArmDataTraining8": lArmDataTraining8,

            "rLegDataTraining1": rLegDataTraining1,
            "rLegDataTraining2": rLegDataTraining2,
            "rLegDataTraining3": rLegDataTraining3,
            "rLegDataTraining4": rLegDataTraining4,
            "rLegDataTraining5": rLegDataTraining5,
            "rLegDataTraining6": rLegDataTraining6,
            "rLegDataTraining7": rLegDataTraining7,
            "rLegDataTraining8": rLegDataTraining8,

            "lLegDataTraining1": lLegDataTraining1,
            "lLegDataTraining2": lLegDataTraining2,
            "lLegDataTraining3": lLegDataTraining3,
            "lLegDataTraining4": lLegDataTraining4,
            "lLegDataTraining5": lLegDataTraining5,
            "lLegDataTraining6": lLegDataTraining6,
            "lLegDataTraining7": lLegDataTraining7,
            "lLegDataTraining8": lLegDataTraining8,

            "chest_lArm_lLegDataTraining1": chest_lArm_lLegDataTraining1,
            "chest_lArm_lLegDataTraining2": chest_lArm_lLegDataTraining2,
            "chest_lArm_lLegDataTraining3": chest_lArm_lLegDataTraining3,
            "chest_lArm_lLegDataTraining4": chest_lArm_lLegDataTraining4,
            "chest_lArm_lLegDataTraining5": chest_lArm_lLegDataTraining5,
            "chest_lArm_lLegDataTraining6": chest_lArm_lLegDataTraining6,
            "chest_lArm_lLegDataTraining7": chest_lArm_lLegDataTraining7,
            "chest_lArm_lLegDataTraining8": chest_lArm_lLegDataTraining8,
            
            ############################3
            "allDataValidation1": allDataValidation1,
            "allDataValidation2": allDataValidation2,
            "allDataValidation3": allDataValidation3,
            "allDataValidation4": allDataValidation4,
            "allDataValidation5": allDataValidation5,
            "allDataValidation6": allDataValidation6,
            "allDataValidation7": allDataValidation7,
            "allDataValidation8": allDataValidation8,

            "chestDataValidation1": chestDataValidation1,
            "chestDataValidation2": chestDataValidation2,
            "chestDataValidation3": chestDataValidation3,
            "chestDataValidation4": chestDataValidation4,
            "chestDataValidation5": chestDataValidation5,
            "chestDataValidation6": chestDataValidation6,
            "chestDataValidation7": chestDataValidation7,
            "chestDataValidation8": chestDataValidation8,

            "rArmDataValidation1": rArmDataValidation1,
            "rArmDataValidation2": rArmDataValidation2,
            "rArmDataValidation3": rArmDataValidation3,
            "rArmDataValidation4": rArmDataValidation4,
            "rArmDataValidation5": rArmDataValidation5,
            "rArmDataValidation6": rArmDataValidation6,
            "rArmDataValidation7": rArmDataValidation7,
            "rArmDataValidation8": rArmDataValidation8, 

            "lArmDataValidation1": lArmDataValidation1,
            "lArmDataValidation2": lArmDataValidation2,
            "lArmDataValidation3": lArmDataValidation3,
            "lArmDataValidation4": lArmDataValidation4,
            "lArmDataValidation5": lArmDataValidation5,
            "lArmDataValidation6": lArmDataValidation6,
            "lArmDataValidation7": lArmDataValidation7,
            "lArmDataValidation8": lArmDataValidation8,

            "rLegDataValidation1": rLegDataValidation1,
            "rLegDataValidation2": rLegDataValidation2,
            "rLegDataValidation3": rLegDataValidation3,
            "rLegDataValidation4": rLegDataValidation4,
            "rLegDataValidation5": rLegDataValidation5,
            "rLegDataValidation6": rLegDataValidation6,
            "rLegDataValidation7": rLegDataValidation7,
            "rLegDataValidation8": rLegDataValidation8,

            "lLegDataValidation1": lLegDataValidation1,
            "lLegDataValidation2": lLegDataValidation2,
            "lLegDataValidation3": lLegDataValidation3,
            "lLegDataValidation4": lLegDataValidation4,
            "lLegDataValidation5": lLegDataValidation5,
            "lLegDataValidation6": lLegDataValidation6,
            "lLegDataValidation7": lLegDataValidation7,
            "lLegDataValidation8": lLegDataValidation8,

            "chest_lArm_lLegDataValidation1": chest_lArm_lLegDataValidation1,
            "chest_lArm_lLegDataValidation2": chest_lArm_lLegDataValidation2,
            "chest_lArm_lLegDataValidation3": chest_lArm_lLegDataValidation3,
            "chest_lArm_lLegDataValidation4": chest_lArm_lLegDataValidation4,
            "chest_lArm_lLegDataValidation5": chest_lArm_lLegDataValidation5,
            "chest_lArm_lLegDataValidation6": chest_lArm_lLegDataValidation6,
            "chest_lArm_lLegDataValidation7": chest_lArm_lLegDataValidation7,
            "chest_lArm_lLegDataValidation8": chest_lArm_lLegDataValidation8,

            }

# Sensor Separation

In [ ]:
for i in range(8):
  print(segments[f"segments_Training{i+1}"][:,:,0:9].shape)
  particular_sensor_data[f"allDataTraining{i+1}"]  = segments[f"segments_Training{i+1}"]
  particular_sensor_data[f"chestDataTraining{i+1}"]  = segments[f"segments_Training{i+1}"][:,:,0:9]
  particular_sensor_data[f"rArmDataTraining{i+1}"]  = segments[f"segments_Training{i+1}"][:,:,9:18]
  particular_sensor_data[f"lArmDataTraining{i+1}"]  = segments[f"segments_Training{i+1}"][:,:,18:27]
  particular_sensor_data[f"rLegDataTraining{i+1}"]  = segments[f"segments_Training{i+1}"][:,:,27:36]
  particular_sensor_data[f"lLegDataTraining{i+1}"]  = segments[f"segments_Training{i+1}"][:,:,36:45]
  particular_sensor_data[f"chest_lArm_lLegDataTraining{i+1}"]  = np.concatenate((particular_sensor_data[f"chestDataTraining{i+1}"]
  , particular_sensor_data[f"lArmDataTraining{i+1}"],particular_sensor_data[f"lLegDataTraining{i+1}"]), axis=2) 

  particular_sensor_data[f"allDataValidation{i+1}"]  = segments[f"segments_Validation{i+1}"]
  particular_sensor_data[f"chestDataValidation{i+1}"]  = segments[f"segments_Validation{i+1}"][:,:,0:9]
  particular_sensor_data[f"rArmDataValidation{i+1}"]  = segments[f"segments_Validation{i+1}"][:,:,9:18]
  particular_sensor_data[f"lArmDataValidation{i+1}"]  = segments[f"segments_Validation{i+1}"][:,:,18:27]
  particular_sensor_data[f"rLegDataValidation{i+1}"]  = segments[f"segments_Validation{i+1}"][:,:,27:36]
  particular_sensor_data[f"lLegDataValidation{i+1}"]  = segments[f"segments_Validation{i+1}"][:,:,36:45]
  particular_sensor_data[f"chest_lArm_lLegDataValidation{i+1}"]  = np.concatenate((particular_sensor_data[f"chestDataValidation{i+1}"]
  , particular_sensor_data[f"lArmDataValidation{i+1}"],particular_sensor_data[f"lLegDataValidation{i+1}"]), axis=2) 



(7980, 125, 9)
(7980, 125, 9)
(7980, 125, 9)
(7980, 125, 9)
(7980, 125, 9)
(7980, 125, 9)
(7980, 125, 9)
(7980, 125, 9)


# Writing into h5 file

In [ ]:
start_time = time.time()

dir = "/content/gdrive/My Drive/Colab Notebooks/HARprojectV2/"
sensors = ["chest", "rArm", "lArm", "lLeg", "rLeg", "chest_lArm_lLeg", "all"]

hf = h5py.File(dir+'HAR_Dataset.h5', 'w')
##########################################################


In [ ]:

for sensor in sensors:
  for i in range(8):
    hf.create_dataset(f'{sensor}DataTrainingSubject{i+1}', data = particular_sensor_data[f"{sensor}DataTraining{i+1}"])
    hf.create_dataset(f'{sensor}LabelsTrainingSubject{i+1}', data = segments[f"labels_Training{i+1}"])

    hf.create_dataset(f'{sensor}DataValidationSubject{i+1}', data = particular_sensor_data[f"{sensor}DataValidation{i+1}"])
    hf.create_dataset(f'{sensor}LabelsValidationSubject{i+1}', data = segments[f"labels_Validation{i+1}"])
    print(sensor)
##########################################################

hf.close()

end_time = time.time()
print(f"total time is: {end_time-start_time}")

chest
chest
chest
chest
chest
chest
chest
chest
rArm
rArm
rArm
rArm
rArm
rArm
rArm
rArm
lArm
lArm
lArm
lArm
lArm
lArm
lArm
lArm
lLeg
lLeg
lLeg
lLeg
lLeg
lLeg
lLeg
lLeg
rLeg
rLeg
rLeg
rLeg
rLeg
rLeg
rLeg
rLeg
chest_lArm_lLeg
chest_lArm_lLeg
chest_lArm_lLeg
chest_lArm_lLeg
chest_lArm_lLeg
chest_lArm_lLeg
chest_lArm_lLeg
chest_lArm_lLeg
all
all
all
all
all
all
all
all
total time is: 57.20213985443115
